In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("nba_games.csv", index_col = 0)

In [7]:
df = df.sort_values("date")

In [8]:
df = df.reset_index(drop = True)

In [9]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [10]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys = False).apply(add_target)

C:\Users\kg200\AppData\Local\Temp\ipykernel_17636\3522436527.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\kg200\AppData\Local\Temp\ipykernel_17636\3522436527.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\kg200\AppData\Local\Temp\ipykernel_17636\3522436527.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

In [11]:
df["target"][pd.isnull(df["target"])] = 2

C:\Users\kg200\AppData\Local\Temp\ipykernel_17636\1618397414.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
C:\Users\kg200\AppData\Local\Temp\ipykernel_17636\1618397414.py:1: SettingWithCopyWarn

In [12]:
df["target"]= df["target"].astype(int, errors = "ignore")

In [13]:
nulls = pd.isnull(df)

In [14]:
nulls = nulls.sum()

In [15]:
nulls = nulls[nulls > 0]

In [16]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [17]:
df = df[valid_columns].copy()

In [18]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha = 1)
split = TimeSeriesSplit(n_splits = 3)

sfs = SequentialFeatureSelector(rr, n_features_to_select = 30, direction = "forward", cv=split)

In [19]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [20]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [22]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [23]:
predictors = list(selected_columns[sfs.get_support()])

In [24]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []

    seasons = sorted(data["season"].unique())

    for i in range (start, len(seasons), step)
        season = seasons[i]

        train = data[data["season"] < season]
        test = data[data["season"] == season]

        model.fit(train[predictors], train["target"], 

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index = test.index)

        combined = pd.concat([test["target"], preds], axis = 1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)

SyntaxError: expected ':' (783807513.py, line 6)

In [ ]:
predictions = backtest(df, rr, predictors)

In [ ]:
from sklearn.metrics import accuracy_score 

predictions = predictions[predictions["actual"]!=2]
accuracy_score(predictions["actual"], predictions["prediction"])

In [ ]:
df.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

In [ ]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [ ]:
def find_team_averages(team):
    rolling = team.rolling(10).mean()

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

In [ ]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]

In [ ]:
df = pd.concat([df, df_rolling], axis = 1)

In [ ]:
df = df.dropna()

In [ ]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

In [ ]:
df = df.copy()

In [ ]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], 
                left_on = ["team", "date_next"], 
                right_on = ["team_opp_next", "date_next"])



In [ ]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [ ]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [ ]:
sfs.fit(full[selected_columns], full["target"])

In [ ]:
predictors = list(selected_columns[sfs.get_support()])

In [ ]:
accuracy_score(predictions["actual"], predictions["prediction"])